In [ ]:
import subprocess
import os
import glob


class Docking_Score:  
    
    global path_content
    global env_content
    
    # 初始化
    def __init__(self,new_seq_batch:list = None,): 
    # 调用Omega_Fold并设置Omega_Fold所需要的路径
        if new_seq_batch is None:
            self.new_seq_batch = []
        elif isinstance(new_seq_batch, list):
            self.new_seq_batch = new_seq_batch
        else:
            raise ValueError("new_seq_batch must be a list")
        
    
        
    def Omega_Fold_Use(self , Fasta_Name = None): 
        # 生成本次建模的fasta，注意的是文件名必须带fasta
        # 输入路径不能为0
        if Fasta_Name is None:  
            raise ValueError("Fasta_Name cannot be None") 
        
        Output_pdb_Path = path_content['pdb_path']
            
        # 生成本次建模的fasta，注意的是文件名必须带fasta    
        Fasta_Path = path_content['fasta_path'] + Fasta_Name + '.fasta'
        Env = env_content['Omega_Fold']
        
        # 定义command：conda run -n Omega_Env omegafold 'Fasta_Path' Output_pdb_Path
        command = ['conda', 'run', '-n', Env, 'omegafold', Fasta_Path, Output_pdb_Path]
        try:  
            result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)  
            print("Omegafold command executed successfully.")     
        except subprocess.CalledProcessError as e:  
            print(f"Omegafold command failed with exit code {e.returncode}.")  
            print("Stderr:", e.stderr.decode()) 
        
        # pdb_old_name = path_content['pdb_path'] + Fasta_Name + ':' + '.pdb'
        # pdb_new_name = path_content['pdb_path'] + Fasta_Name + '.pdb'
        # os.rename(pdb_old_name, pdb_new_name)  
        # print(f"文件已重命名为：{pdb_new_name}")

    #设置pydock3所需要的环境变量和变量名声明       
    def Pydock3_Path_Alias(self):
        original_path = os.environ['PATH']  
        additional_paths = path_content['pydock3_need_path']
        additional_paths = additional_paths.split(' ')
        new_path = os.pathsep.join(additional_paths + original_path.split(os.pathsep))
        Alias_path = path_content['alias_sh_path']
        try:  
            result = subprocess.run(['bash', Alias_path], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)  
            print("Shell script executed successfully.")  
            print(result.stdout.decode())  
        except subprocess.CalledProcessError as e:  
            print(f"Shell script failed with exit code {e.returncode}.")  
            print("Stderr:", e.stderr.decode())
            
    def Pydock3_set_ini(self, ligand_Name = None, receptor_Name = None, receptor_Path = None,receptor_mol = 'A', receptor_newmol = 'B'):
        #默认受体的链名为F，新生成的受体链名为B。到时候看一下是否需要设置一个receptor_list.csv记录这个。
        # 确保ini_template,ligand_Name和receptor_Name不为None，否则后续路径拼接会出错  
        if ligand_Name is None: 
            raise ValueError("ligand_Name and receptor_Name cannot be None")  
        if receptor_Name is None: 
            raise ValueError("ligand_Name and receptor_Name cannot be None")
        if receptor_Path is None: 
            raise ValueError("receptor_Path cannot be None")
        
        #配体和受体的信息，受体只输入一个名字，所以要加.pdb，
        ligand_data = {'ligand_name' : path_content['ligand_path'] + ligand_Name + '.pdb' , 
                       'receptor_name' : receptor_Path,
                       'receptor_mol' : receptor_mol,
                       'receptor_newmol' : receptor_newmol
                        }  
  
        # 指定INI文件的路径为temp缓存文件中的docking部分，缓存可能需要移除
        ini_file_path = path_content['ini_file_path'] + ligand_Name+ '_' + receptor_Name + '.ini'
  
        # 使用with语句打开文件（如果不存在则创建），并写入内容  
        with open(ini_file_path, 'w') as ini_file:  
            ini_file.write(ini_template.format(**ligand_data)) 
            
  
        print(f"INI文件 '{ini_file_path}' 已成功创建并写入内容。")

    def Pydock3_ini_setup(self, ini_Name = None):
        #设置重试参数
        _retry_times = 0
        if ini_Name is None:  
            raise ValueError("ini_Name cannot be None")
        # 使用conda run在Pydock3_try环境中运行pydock3和zdock
        #pydock_name_setup = ['conda', 'run', '-n', 'Pydock3_try','pydock3', 'RSV','setup']
        #conda run -n Pydock3_try ../../123/pyDock3/pyDock3/pyDock3 ./temp/docking/try setup
        pydock_path = path_content['pydock3_software_path']
        ini_Name = path_content['ini_file_path'] + ini_Name
        Env = env_content['Pydock3']
        #设置指令名
        pydock_name_setup = ['conda', 'run', '-n', Env ,'bash','-c', f'{pydock_path} {ini_Name} setup']  
        while _retry_times <10000:
            try:
                result = subprocess.run(pydock_name_setup, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                print("pydock_name_setup executed successfully.")
                _retry_times = 0
                break
            except subprocess.CalledProcessError as e:
                print(f"pydock_name_setup failed with exit code {e.returncode}.")
                print("Stderr:", e.stderr.decode())
                _retry_times += 1
        if _retry_times >= 10000:
            print("pydock_name_setup failed after 10 retries.")
            _retry_times = 0
            
    def zdock_make_sur_generate(self,ligand_Name = None,receptor_Path = None):
        #ligand_Name不能为None
        if ligand_Name is None:  
            raise ValueError("ligand_Name cannot be None") 
        if receptor_Path is None:  
            raise ValueError("receptor_Path cannot be None") 
        #传递需要的参数
        receptor_Path = path_content['ini_file_path'] + ligand_Name +'_rec.pdb'
        ligand_Path = path_content['ini_file_path'] + ligand_Name + '_lig.pdb'
        Env = env_content['Pydock3']
        #使用参数构建zdock mark_sur命令
        mark_sur_command1 = ['conda', 'run', '-n', Env, 'bash', '-c', f'mark_sur {receptor_Path}'] 
        mark_sur_command2 = ['conda', 'run', '-n', Env, 'bash', '-c', f'mark_sur {ligand_Path}'] 
        try:  
        # 使用 shell=True 来允许 shell 重定向  
            subprocess.check_call(mark_sur_command1, shell=True)  
            print("mark_sur_command1 executed successfully") 
            subprocess.check_call(mark_sur_command2, shell=True)  
            print("mark_sur_command2 executed successfully") 
        except subprocess.CalledProcessError as e:  
            print(f"mark_sur_command failed with exit code {e.returncode}.")

    def zdock_parallel_run(self, docking_list:list = None):
        #并行运行zdock，注意docking_list的输入应为列表，内容为名字。并行数默认为27，int
        #docking_list不能为空
        # parallel_Num = 63 = 7 * 9
        if docking_list is None:  
            raise ValueError("docking_list cannot be None") 
            
        # parallel_num 储存在config文件中
        if config_content["parallel_num"]/len(receptor) != len(docking_list):  
            raise ValueError("Parallel numbers doesn't pair") 
            
        #设定并行运算的模版
        zdock_parallel_command = 'parallel -j $(nproc) ::: '
        zdock_parallel_subunit_template = " '{} -R {} -L {} -o {} -N 1' "
        
        #初始化生成并行运算指令的for循环变量
        receptor = ''
        ligand = ''
        output_name = ''
        zdock_parallel_subunit = ''
        zdock_path = path_content['zdock_software_path']
        
        #编辑并行运算的指令
        # for i in receptor_path:
        #     for x in docking_list:
        #         receptor = i + '_rec.pdb'
        #         ligand = path_content['ini_file_path'] + x + '_lig.pdb'
        #         output_name = path_content['ini_file_path'] + x + '_' + i
        #         zdock_parallel_subunit = zdock_parallel_subunit_template.format(zdock_path,receptor,ligand,output_name)
        #         zdock_parallel_command += zdock_parallel_subunit
        
        for x in docking_list:
            receptor = path_content['ini_file_path'] + x + '_rec.pdb'
            ligand = path_content['ini_file_path'] + x + '_lig.pdb'
            output_name = path_content['ini_file_path'] + x
            zdock_parallel_subunit = zdock_parallel_subunit_template.format(zdock_path,receptor,ligand,output_name)
            zdock_parallel_command += zdock_parallel_subunit

        print(zdock_parallel_command)

        print(zdock_parallel_command)
        
        try:  
        # 使用 shell=True 来允许 shell 重定向  
            subprocess.check_call(zdock_parallel_command, shell=True)  
            print("zdock_parallel is executing") 
        except subprocess.CalledProcessError as e:  
            print(f"zdock_parallel failed with exit code {e.returncode}.")
        
    def Pydock3_zdock_rot(self, rot_Name = None):
        _retry_times = 0
        #rot_Name不能为空
        if rot_Name is None:  
            raise ValueError("rot_Name cannot be None")
        Env = env_content['Pydock3']
        
        #编辑rot需要的命令
        pydock3_path = path_content['pydock3_software_path']
        pydock3_rot_Name = path_content['ini_file_path'] + rot_Name
        rot_template = ['conda','run','-n', Env ,'bash','-c', f'{pydock3_path} {pydock3_rot_Name} rotzdock']
        print(rot_template)
    
        while _retry_times <10000:
            try:
                result = subprocess.run(rot_template, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                print("pydock3_rot_command executed successfully.")
                _retry_times = 0
                break
            except subprocess.CalledProcessError as e:
                print(f"pydock3_rot_command failed with exit code {e.returncode}.")
                print("Stderr:", e.stderr.decode())
                _retry_times += 1
        if _retry_times >= 10000:
            print("pydock3_rot_command failed after 10 retries.")
            _retry_times = 0
    
    @staticmethod
    def add_zdock_extension():  
        """  
        遍历指定目录下的所有文件，找到没有后缀名的文件，并给它们添加.zdock后缀。  

        :param directory: 指定目录的路径  
        """  
        # 遍历目录中的所有文件和文件夹 
        directory = path_content['ini_file_path']
        for filename in os.listdir(directory):  
            # 构建完整的文件路径  
            file_path = os.path.join(directory, filename)  

            # 检查是否是文件（排除目录）  
            if os.path.isfile(file_path):  
                # 检查文件是否有后缀名（这里简单地通过是否包含'.'来判断）  
                if '.' not in filename:  
                    # 如果没有后缀名，则添加.zdock后缀  
                    new_filename = filename + '.zdock'  
                    new_file_path = os.path.join(directory, new_filename)  

                    # 重命名文件  
                    os.rename(file_path, new_file_path)  
                    print(f"Renamed '{file_path}' to '{new_file_path}'")
        
        
        
    def Pydock3_rot_dockser_parallel(self,dockser_Name = None):
        _retry_times = 0
        #rot_Name不能为空
        if dockser_Name is None:  
            raise ValueError("dockser_Name cannot be None")
        Env = env_content['Pydock3']
        
        #编辑rot需要的命令
        pydock3_path = path_content['pydock3_software_path']
        pydock3_dockser_Name = path_content['ini_file_path'] + dockser_Name
        dockser_template = ['conda','run','-n', Env ,'bash','-c', f'{pydock3_path} {pydock3_dockser_Name} dockser']
        print(dockser_template)
    
        while _retry_times <10000:
            try:
                result = subprocess.run(dockser_template, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                print("pydock3_dockser_command executed successfully.")
                _retry_times = 0
                break
            except subprocess.CalledProcessError as e:
                print(f"pydock3_dockser_command failed with exit code {e.returncode}.")
                print("Stderr:", e.stderr.decode())
                _retry_times += 1
        if _retry_times >= 10000:
            print("pydock3_dockser_command failed after 10 retries.")
            _retry_times = 0
            
            
    def dock_and_score_run(self):            
        names = [seq.name for seq in self.new_seq_batch]
        deal_Names = []
        a = ''
        for i in names:
            for x in receptor_key:
                a = i + '_' + x
                deal_Names.append(a)
        #先生成全部的pdb并且删除冒号
        for i in names:
            self.Omega_Fold_Use(Fasta_Name = i)
            print(f'Omegafold 本次生成了{i}')
        
        self.Pydock3_Path_Alias()
        
        #写在docking文件夹中写入ini文件
        for i in names:
            for index, receptor_name in enumerate(receptor_key):  
                self.Pydock3_set_ini(ligand_Name=i, receptor_Path=receptor_path[index], receptor_Name=receptor_name)
        
        # 然后根据deal_Names中的名字对ini进行setup
        for i in deal_Names:
            self.Pydock3_ini_setup(ini_Name = i)
            self.zdock_make_sur_generate(ligand_Name = i , receptor_Path = i)
        
        #然后跑并行
        print(deal_Names)
        self.zdock_parallel_run(docking_list_jerry = deal_Names)
        self.add_zdock_extension()
        for i in deal_Names:
            self.Pydock3_zdock_rot(rot_Name = deal_Names)
            self.Pydock3_rot_dockser_parallel(dockser_Name = deal_Names)
        
        ene_path = path_content['ini_fild_path']
        ene_dict = {}
        ene_list = []
        for i in names:
            ene_list = glob.glob(ene_path + f"/*{i}*.ene")
            ene_dict[i] = ene_list
        
        return ene_dict

In [ ]:
import os
import subprocess
!pwd
os.chdir('../Evolution_dock')
!pwd

In [ ]:
os
docking_score= Docking_Score()
help(Docking_Score.zdock_parallel_run)
docking_score.zdock_parallel_run(docking_list = ['alias00000001_MERS', 'alias00000001_SAR', 'alias00000001_RSV', 'alias00000001_SARS', 'alias00000001_IVA', 'alias00000001_229E', 'alias00000001_OC43', 'alias00000001_NL63', 'alias00000001_PIV', 'alias00000002_MERS', 'alias00000002_SAR', 'alias00000002_RSV', 'alias00000002_SARS', 'alias00000002_IVA', 'alias00000002_229E', 'alias00000002_OC43', 'alias00000002_NL63', 'alias00000002_PIV'])